# Warehouse Problem - Lagrangian relaxation

Let's recall the MIP formulation:

\begin{align*}
\min &\qquad \sum_w c_w x_w + \sum_{w ,c} t_{w,c} y_{w,c}  & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w,c \\
&\sum_w y_{w,c} = 1 & \forall c \\
&\sum_c y_{w,c} \leq C_w  x_w & \forall w \\
&x_w, y_{w,c}, z_c \in \mathbb{B} & \forall w,c
\end{align*}

In [1]:
fixed = 30  # c_w (all the costs are the same)
M = 10*fixed  # the penalty
capacity = [1,4,2,1,3]  # the capacity of a warehouse
supplyCost = [[20,24,11,25,30],  # t_{w,c}
              [28,27,82,83,74],
              [74,97,71,96,70],
              [2,55,73,69,61],
              [46,96,59,83,4],
              [42,22,29,67,59],
              [1,5,73,59,56],
              [10,73,13,43,96],
              [93,35,63,85,46],
              [47,65,55,71,95]]
nbStores = len(supplyCost)
nbWarehouses = len(capacity)
Stores = range(nbStores)
Warehouses = range(nbWarehouses)

### Excercices

#### 1. Find the lagrangian relaxation formulation where the constraints 2 are relaxed.

#### 2. Implement the lagrangian relaxation.

In [2]:
from docplex.mp.model import Model

def lagrangian_relaxation(rlambda=[1]*nbStores):
    model = Model(name='lagrangian')
    
     # Decision variables
    x = {w: model.binary_var(name=f'x_{w}') for w in Warehouses}  # 1 if warehouse w is open
    y = {(w, s): model.binary_var(name=f'y_{w}_{s}') for w in Warehouses for s in Stores}  # 1 if customer s is served by warehouse w
    
    # Lagrangian objective function
    Lagrangian_obj = sum(fixed * x[w] for w in Warehouses) + \
                     sum(supplyCost[s][w] * y[w, s] for w in Warehouses for s in Stores) + \
                     sum(rlambda[s] * (1 - sum(y[w, s] for w in Warehouses)) for s in Stores)

    # Constraints: if a warehouse is not open, it cannot supply any store
    for w in range(nbWarehouses):
        for s in range(nbStores):
            model.add_constraint(y[w, s] <= x[w], f'supply_{s}_{w}')  # If warehouse w is closed, it cannot serve any store

    # Constraints: each warehouse cannot serve more stores than its capacity
    for w in range(nbWarehouses):
        model.add_constraint(model.sum(y[w, s] for s in range(nbStores)) <= capacity[w] * x[w], f'capacity_{w}')  # Respect warehouse capacity

    
    # Set the objective
    model.minimize(Lagrangian_obj)

    # Solve the model
    solution = model.solve()
    
    # Print the solution
    if solution:
        print(f"Sub Objective Value: {solution.objective_value:.2f}.")
      
    else:
        print("No feasible solution found for the Lagrangian Objective function.")
    
    relaxation_constraints = [1 - sum(y[w, s].solution_value for w in Warehouses) for s in Stores]
    
    x_sol = [x[w].solution_value for w in Warehouses]
    y_sol = {(w, s): y[w,s].solution_value for w in Warehouses for s in Stores}
    
    return (model.objective_value,  # the objective value
            solution, # solution
            relaxation_constraints,
            x_sol,
            y_sol)  # slack of the relaxed constraints

#### Try the function

In [3]:
obj, msol, relaxed, x_sol, y_sol = lagrangian_relaxation([fixed]*nbStores)
print(msol, relaxed)

Sub Objective Value: 282.00.
solution for: lagrangian
objective: 282
status: OPTIMAL_SOLUTION(2)
x_1=1
x_2=1
y_1_0=1
y_1_1=1
y_1_5=1
y_1_6=1
y_2_0=1
y_2_7=1
 [-1.0, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 1, 1]


#### 3. Implement the Subgradient Method



In [7]:
rlambda = [fixed]*nbStores
fsol = None
iteration = 0
step = 1
LB = 0
UB=M*nbStores
bestFeasibleValue = UB
bestFeasible = 0

while True:

    print("-"*40)
    print(f"Iteration # {iteration+1}")
    sub_obj, msol, relaxation_constraints, x_sol, y_sol = lagrangian_relaxation(rlambda)
    
    #UB = max(UB, sub_obj)
    is_feasible = all(abs(slack) < 1e-5 for slack in relaxation_constraints)

    if is_feasible:
        currentFeasibleValue = sub_obj

        if (abs(currentFeasibleValue - bestFeasibleValue)< 1e-5):
            break
            
        if currentFeasibleValue < bestFeasibleValue:
            bestFeasibleValue = currentFeasibleValue
     
    # Update Lagrange multipliers using element-wise multiplication
    rlambda = [rlambda[i] + step * relaxation_constraints[i] for i in range(len(rlambda))]
    
    iteration += 1

  
print('========================')
print('Lagrangian dual: ' + str(UB))
    
if bestFeasible != 0:
    print('-- best feasible --')
    print(bestFeasible)

----------------------------------------
Iteration # 1
Sub Objective Value: 282.00.
----------------------------------------
Iteration # 2
Sub Objective Value: 288.00.
----------------------------------------
Iteration # 3
Sub Objective Value: 294.00.
----------------------------------------
Iteration # 4
Sub Objective Value: 299.00.
----------------------------------------
Iteration # 5
Sub Objective Value: 304.00.
----------------------------------------
Iteration # 6
Sub Objective Value: 308.00.
----------------------------------------
Iteration # 7
Sub Objective Value: 311.00.
----------------------------------------
Iteration # 8
Sub Objective Value: 313.00.
----------------------------------------
Iteration # 9
Sub Objective Value: 315.00.
----------------------------------------
Iteration # 10
Sub Objective Value: 317.00.
----------------------------------------
Iteration # 11
Sub Objective Value: 319.00.
----------------------------------------
Iteration # 12
Sub Objective Valu